<a href="https://colab.research.google.com/github/chettkulkarni/MachineLearning/blob/master/weatherDataLoad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import pandas as pd

In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

link = 'https://drive.google.com/open?id=1ApRYxyMqnFKH8XW1GbQ-KZNJdbdeEdV9'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='


downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Fire.csv')

1ApRYxyMqnFKH8XW1GbQ-KZNJdbdeEdV9


In [0]:
#Api to fetch the records

# response = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&startdate=2017-01-01&enddate=2017-01-31&locationid=FIPS:23&format=csv", headers={'token': 'RGFmBkMzqCUgFXbiQWeqalQIAAoBMhTQ'})
# response = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/locationcategories", headers={'token': 'RGFmBkMzqCUgFXbiQWeqalQIAAoBMhTQ'})

# response.content

In [0]:
# responseObject=response.json()

In [0]:
# df=pd.DataFrame.from_dict(responseObject['results'])

In [2]:
# df
files = [str(x)+'.csv.gz' for x in range(1993,2010)]
files

['1993.csv.gz',
 '1994.csv.gz',
 '1995.csv.gz',
 '1996.csv.gz',
 '1997.csv.gz',
 '1998.csv.gz',
 '1999.csv.gz',
 '2000.csv.gz',
 '2001.csv.gz',
 '2002.csv.gz',
 '2003.csv.gz',
 '2004.csv.gz',
 '2005.csv.gz',
 '2006.csv.gz',
 '2007.csv.gz',
 '2008.csv.gz',
 '2009.csv.gz']

In [161]:
#fetching data from ftp server for years from 1993 to 2000

from ftplib import FTP
from datetime import datetime

start = datetime.now()
ftp = FTP('ftp.ncdc.noaa.gov')
ftp.login()
ftp.cwd("pub/data/ghcn/daily/by_year/")


# Get All Files
files = [str(x)+'.csv.gz' for x in range(1993,2010)]

# Print out the files
for file in files:
	print("Downloading..." + file)
	ftp.retrbinary("RETR " + file ,open(file, 'wb').write)

ftp.close()

end = datetime.now()
diff = end - start
print('All files downloaded for ' + str(diff.seconds) + 's')

Downloading...1993.csv.gz
Downloading...1994.csv.gz
Downloading...1995.csv.gz
Downloading...1996.csv.gz
Downloading...1997.csv.gz
Downloading...1998.csv.gz
Downloading...1999.csv.gz
Downloading...2000.csv.gz
Downloading...2001.csv.gz
Downloading...2002.csv.gz
Downloading...2003.csv.gz
Downloading...2004.csv.gz
Downloading...2005.csv.gz
Downloading...2006.csv.gz
Downloading...2007.csv.gz
Downloading...2008.csv.gz
Downloading...2009.csv.gz
All files downloaded for 522s


In [0]:
# df = pd.read_csv('1987.csv.gz', compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)

In [0]:
# df

In [0]:
#getting ready with sparkx instance

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
!pip install pyspark
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkFiles
import pyspark.sql.functions as func

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
#generating spark contect and sessions to consume our datasets

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("gfcj") \
        .getOrCreate()

In [0]:
#adding all the files to spark context to spread them into rdd and perform faster exexution on huge dataset

In [0]:
sc =SparkContext.getOrCreate()

for file in files:
  sc.addFile(file)

In [0]:
sqlContext = SQLContext(sc)
sqlContext
# df = sqlContext.read.csv(SparkFiles.get(files), header=True, inferSchema= True)
# df.printSchema()

spark.read.format("csv").option("header", "true").load("*.csv.gz")
df=spark.read.option("header", "false").csv("*.csv.gz")

In [8]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [9]:
df.show(5)

+-----------+--------+----+----+----+----+---+----+
|        _c0|     _c1| _c2| _c3| _c4| _c5|_c6| _c7|
+-----------+--------+----+----+----+----+---+----+
|US1FLSL0019|20090101|PRCP|   0|null|null|  N|null|
|US1FLSL0019|20090101|SNOW|   0|null|null|  N|null|
|ASN00037003|20090101|PRCP| 150|null|null|  a|null|
|USC00178998|20090101|TMAX|-111|null|null|  0|1800|
|USC00178998|20090101|TMIN|-183|null|null|  0|1800|
+-----------+--------+----+----+----+----+---+----+
only showing top 5 rows



In [0]:
sc.addFile('ghcnd-stations.csv')

In [0]:
df2=spark.read.option("header", "true").csv('ghcnd-stations.csv')

In [12]:
df2

DataFrame[id: string, lat: string, long: string, _c3: string, _c4: string, _c5: string]

In [13]:
df.show(5)

+-----------+--------+----+----+----+----+---+----+
|        _c0|     _c1| _c2| _c3| _c4| _c5|_c6| _c7|
+-----------+--------+----+----+----+----+---+----+
|US1FLSL0019|20090101|PRCP|   0|null|null|  N|null|
|US1FLSL0019|20090101|SNOW|   0|null|null|  N|null|
|ASN00037003|20090101|PRCP| 150|null|null|  a|null|
|USC00178998|20090101|TMAX|-111|null|null|  0|1800|
|USC00178998|20090101|TMIN|-183|null|null|  0|1800|
+-----------+--------+----+----+----+----+---+----+
only showing top 5 rows



In [14]:
df2.show(5)

+-----------+-------+--------+----+----+----+
|         id|    lat|    long| _c3| _c4| _c5|
+-----------+-------+--------+----+----+----+
|ACW00011604|17.1167|-61.7833|null|null|null|
|ACW00011647|17.1333|-61.7833|null|null|null|
|AE000041196| 25.333|  55.517|null|null|null|
|AEM00041194| 25.255|  55.364|null|null|null|
|AEM00041217| 24.433|  54.651|null|null|null|
+-----------+-------+--------+----+----+----+
only showing top 5 rows



In [0]:
data=df.selectExpr('_c0 as id','_c1 as date','_c2 as type','_c3 as value')

In [16]:
data.show()

+-----------+--------+----+-----+
|         id|    date|type|value|
+-----------+--------+----+-----+
|US1FLSL0019|20090101|PRCP|    0|
|US1FLSL0019|20090101|SNOW|    0|
|ASN00037003|20090101|PRCP|  150|
|USC00178998|20090101|TMAX| -111|
|USC00178998|20090101|TMIN| -183|
|USC00178998|20090101|TOBS| -139|
|USC00178998|20090101|PRCP|    0|
|USC00178998|20090101|SNOW|    0|
|USC00178998|20090101|SNWD|    0|
|USC00242347|20090101|TMAX|  -17|
|USC00242347|20090101|TMIN|  -72|
|USC00242347|20090101|TOBS|  -44|
|USC00242347|20090101|PRCP|   23|
|USC00242347|20090101|SNOW|   38|
|NOE00133566|20090101|TMAX|   12|
|NOE00133566|20090101|TMIN|  -37|
|NOE00133566|20090101|PRCP|   80|
|NOE00133566|20090101|SNWD|   30|
|US1SCPC0003|20090101|PRCP|    0|
|US1SCPC0003|20090101|SNOW|    0|
+-----------+--------+----+-----+
only showing top 20 rows



In [17]:
df2.show(2)

+-----------+-------+--------+----+----+----+
|         id|    lat|    long| _c3| _c4| _c5|
+-----------+-------+--------+----+----+----+
|ACW00011604|17.1167|-61.7833|null|null|null|
|ACW00011647|17.1333|-61.7833|null|null|null|
+-----------+-------+--------+----+----+----+
only showing top 2 rows



In [0]:
final_df = data.join(df2, on=['id'], how='inner')

In [0]:
final_df=final_df.selectExpr('id','lat as LATITUDE','long as LONGITUDE','date','type','value')

In [0]:
final_df.registerTempTable("temperature")

In [0]:
# def spark_shape(self):
#     return (self.count(), len(self.columns))
# pyspark.sql.dataframe.DataFrame.shape = spark_shape

In [0]:
# final_df.shape()

In [0]:
# df.shape()

In [177]:
fire=pd.read_csv('Fire.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,11,12,13,14,15,16,17,18,19,36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df=fire

df['disc_clean_date'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
#Convert containment date from julian to standard date
df['cont_clean_date'] = pd.to_datetime(df['CONT_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
#Day of month string
df['discovery_month'] = df['disc_clean_date'].dt.strftime('%b')
#Returns the weekday string
df['discovery_weekday'] = df['disc_clean_date'].dt.strftime('%a')
#Merge discovery date and time 


In [179]:
fire_filter=[1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010]
df_1993=df[df.FIRE_YEAR.isin(fire_filter)]

fire_filter

[1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010]

In [0]:
df_1993=df_1993[['LATITUDE','LONGITUDE','cont_clean_date']]

In [0]:
df_1993=df_1993.round()

In [0]:
df_1993=df_1993.dropna()

In [183]:
df_1993

,LATITUDE,LONGITUDE,cont_clean_date
0,40.0,-121.0,2005-02-02
1,39.0,-120.0,2004-05-12
2,39.0,-121.0,2004-05-31
3,39.0,-120.0,2004-07-03
4,39.0,-120.0,2004-07-03
...,...,...,...
1880399,34.0,-117.0,2009-05-03
1880409,39.0,-121.0,2009-08-19
1880411,36.0,-119.0,2009-06-19
1880417,38.0,-120.0,2010-09-15


In [0]:
# type(df_1993['cont_clean_date'])

In [0]:
df_1993['cont_clean_date']=df_1993['cont_clean_date'].astype(str)

In [0]:
df_1993['date']=df_1993['cont_clean_date'].replace('-', '', regex=True).astype(int)

In [187]:
df_1993

,LATITUDE,LONGITUDE,cont_clean_date,date
0,40.0,-121.0,2005-02-02,20050202
1,39.0,-120.0,2004-05-12,20040512
2,39.0,-121.0,2004-05-31,20040531
3,39.0,-120.0,2004-07-03,20040703
4,39.0,-120.0,2004-07-03,20040703
...,...,...,...,...
1880399,34.0,-117.0,2009-05-03,20090503
1880409,39.0,-121.0,2009-08-19,20090819
1880411,36.0,-119.0,2009-06-19,20090619
1880417,38.0,-120.0,2010-09-15,20100915


In [0]:
df_1993=df_1993[['LATITUDE','LONGITUDE','date']]

In [0]:
df_1993.to_csv(r'fire_1993.csv')

In [0]:
sc.addFile('fire_1993.csv')

In [0]:
fire=spark.read.option("header", "true").csv('fire_1993.csv')

In [23]:
fire.show(5)

+---+--------+---------+--------+
|_c0|LATITUDE|LONGITUDE|    date|
+---+--------+---------+--------+
|  0|    40.0|   -121.0|20050202|
|  1|    39.0|   -120.0|20040512|
|  2|    39.0|   -121.0|20040531|
|  3|    39.0|   -120.0|20040703|
|  4|    39.0|   -120.0|20040703|
+---+--------+---------+--------+
only showing top 5 rows



In [24]:
final_df.show()

+-----------+--------+---------+--------+----+-----+
|         id|LATITUDE|LONGITUDE|    date|type|value|
+-----------+--------+---------+--------+----+-----+
|US1FLSL0019| 27.3237| -80.3111|20090101|PRCP|    0|
|US1FLSL0019| 27.3237| -80.3111|20090101|SNOW|    0|
|ASN00037003|-20.4661| 138.4683|20090101|PRCP|  150|
|USC00178998| 44.6917| -68.3475|20090101|TMAX| -111|
|USC00178998| 44.6917| -68.3475|20090101|TMIN| -183|
|USC00178998| 44.6917| -68.3475|20090101|TOBS| -139|
|USC00178998| 44.6917| -68.3475|20090101|PRCP|    0|
|USC00178998| 44.6917| -68.3475|20090101|SNOW|    0|
|USC00178998| 44.6917| -68.3475|20090101|SNWD|    0|
|USC00242347| 47.3181|-109.9372|20090101|TMAX|  -17|
|USC00242347| 47.3181|-109.9372|20090101|TMIN|  -72|
|USC00242347| 47.3181|-109.9372|20090101|TOBS|  -44|
|USC00242347| 47.3181|-109.9372|20090101|PRCP|   23|
|USC00242347| 47.3181|-109.9372|20090101|SNOW|   38|
|NOE00133566| 62.1031|   5.5817|20090101|TMAX|   12|
|NOE00133566| 62.1031|   5.5817|20090101|TMIN|

In [0]:
final_df=final_df.withColumn("LATITUDE", func.round(final_df["LATITUDE"]))
final_df=final_df.withColumn("LONGITUDE", func.round(final_df["LONGITUDE"]))

In [26]:
final_df.show()

+-----------+--------+---------+--------+----+-----+
|         id|LATITUDE|LONGITUDE|    date|type|value|
+-----------+--------+---------+--------+----+-----+
|US1FLSL0019|    27.0|    -80.0|20090101|PRCP|    0|
|US1FLSL0019|    27.0|    -80.0|20090101|SNOW|    0|
|ASN00037003|   -20.0|    138.0|20090101|PRCP|  150|
|USC00178998|    45.0|    -68.0|20090101|TMAX| -111|
|USC00178998|    45.0|    -68.0|20090101|TMIN| -183|
|USC00178998|    45.0|    -68.0|20090101|TOBS| -139|
|USC00178998|    45.0|    -68.0|20090101|PRCP|    0|
|USC00178998|    45.0|    -68.0|20090101|SNOW|    0|
|USC00178998|    45.0|    -68.0|20090101|SNWD|    0|
|USC00242347|    47.0|   -110.0|20090101|TMAX|  -17|
|USC00242347|    47.0|   -110.0|20090101|TMIN|  -72|
|USC00242347|    47.0|   -110.0|20090101|TOBS|  -44|
|USC00242347|    47.0|   -110.0|20090101|PRCP|   23|
|USC00242347|    47.0|   -110.0|20090101|SNOW|   38|
|NOE00133566|    62.0|      6.0|20090101|TMAX|   12|
|NOE00133566|    62.0|      6.0|20090101|TMIN|

In [0]:
complete = fire.join(final_df, on=['LATITUDE','LONGITUDE','date'], how='inner')

In [0]:
complete.show(1)

In [0]:
complete